In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46699
2,Huelva,Confirmados PDIA 14 días,518
3,Huelva,Tasa PDIA 14 días,"100,94120856636202"
4,Huelva,Confirmados PDIA 7 días,292
5,Huelva,Tasa PDIA 7 dias,"56,90122181733149"
6,Huelva,Total Confirmados,46902
7,Huelva,Curados,45467
8,Huelva,Fallecidos,421


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46699.0


/tmp/ipykernel_41479/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12854.0


/tmp/ipykernel_41479/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_41479/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_41479/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_41479/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1410 personas en los últimos 7 días 

Un positivo PCR cada 882 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46699.0,292.0,518.0,513170.0,56.901222,100.941209,111.0
Huelva-Costa,27601.0,165.0,282.0,289548.0,56.985370,97.393178,65.0
Condado-Campiña,14540.0,105.0,196.0,156231.0,67.208173,125.455255,36.0
Huelva (capital),12854.0,102.0,163.0,143837.0,70.913604,113.322719,36.0
Cartaya,2185.0,31.0,60.0,20083.0,154.359408,298.760145,20.0
Palos de la Frontera,1122.0,30.0,51.0,11742.0,255.493102,434.338273,14.0
Sierra de Huelva-Andévalo Central,4130.0,20.0,38.0,67391.0,29.677553,56.387351,9.0
Bollullos Par del Condado,1326.0,15.0,28.0,14387.0,104.260791,194.620143,6.0
Isla Cristina,3189.0,14.0,28.0,21393.0,65.441967,130.883934,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,59.0,1.0,7.0,713.0,140.252454,981.767181,0.0
Palos de la Frontera,1122.0,30.0,51.0,11742.0,255.493102,434.338273,14.0
Granada de Río-Tinto (La),6.0,1.0,1.0,243.0,411.522634,411.522634,NaN
Almonaster la Real,92.0,2.0,6.0,1788.0,111.856823,335.570470,0.0
Cartaya,2185.0,31.0,60.0,20083.0,154.359408,298.760145,20.0
San Bartolomé de la Torre,393.0,5.0,8.0,3761.0,132.943366,212.709386,3.0
Bollullos Par del Condado,1326.0,15.0,28.0,14387.0,104.260791,194.620143,6.0
Moguer,2059.0,18.0,36.0,21867.0,82.315818,164.631637,3.0
Fuenteheridos,14.0,1.0,1.0,709.0,141.043724,141.043724,NaN


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Campofrío,59.0,1.0,7.0,713.0,140.252454,981.767181,0.0,0.142857
San Juan del Puerto,770.0,1.0,5.0,9411.0,10.625863,53.129317,0.0,0.200000
Almonaster la Real,92.0,2.0,6.0,1788.0,111.856823,335.570470,0.0,0.333333
Trigueros,546.0,1.0,3.0,7862.0,12.719410,38.158229,0.0,0.333333
Bonares,469.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Rociana del Condado,676.0,1.0,3.0,7939.0,12.596045,37.788135,0.0,0.333333
Almonte,2354.0,11.0,28.0,24507.0,44.885135,114.253071,2.0,0.392857
Aljaraque,1653.0,5.0,11.0,21474.0,23.283971,51.224737,3.0,0.454545
Ayamonte,2114.0,2.0,4.0,21104.0,9.476876,18.953753,1.0,0.500000
